In [1]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()

ml_client = MLClient.from_config(credential=credential)
print("Conectado ao workspace:", ml_client.workspace_name)

Found the config file in: .\config.json
Class DeploymentTemplateOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Conectado ao workspace: workspace-dioDS


In [5]:
%%writefile src/sorvetes-training.py

import pandas as pd
import numpy as np
import mlflow
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

print("Loading Data...")
sorvetes = pd.read_csv('Vendas de Sorvete.csv', sep=';')  # ← fix aqui

print("Colunas:", sorvetes.columns.tolist())
print(sorvetes.head())

X = sorvetes[['Temperatura (°C)']].values
y = sorvetes['Vendas (unidades)'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

mlflow.autolog()

model = LinearRegression()
model.fit(X_train, y_train)

y_hat = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_hat))
r2   = r2_score(y_test, y_hat)

print(f"RMSE: {rmse:.2f}")
print(f"R²:   {r2:.4f}")

Overwriting src/sorvetes-training.py


In [6]:
from azure.ai.ml import command

job = command(
    code="./src",
    command="python sorvetes-training.py",
    environment="AzureML-sklearn-1.0-ubuntu20.04-py38-cpu:11",
    compute="sorvete",
    display_name="Sorvete_ML",
    experiment_name="Sorvetes"
)

returned_job = ml_client.create_or_update(job)
print(returned_job)
print("Link: ", returned_job.studio_url)

Uploading src (0.0 MBs): 100%|##########| 2773/2773 [00:00<00:00, 3398.11it/s]




type: command
outputs:
  default:
    mode: rw_mount
    type: uri_folder
    path: azureml://datastores/workspaceartifactstore/ExperimentRun/dcid.boring_horse_t05w9nm6pw
environment: azureml:AzureML-sklearn-1.0-ubuntu20.04-py38-cpu:11
resources:
  instance_count: 1
  shm_size: 2g
component:
  name: boring_horse_t05w9nm6pw
  display_name: Sorvete_ML
  type: command
  outputs:
    default:
      type: uri_folder
      mode: rw_mount
  command: python sorvetes-training.py
  environment: azureml:AzureML-sklearn-1.0-ubuntu20.04-py38-cpu:11
  code: azureml:/subscriptions/37a4025b-42d4-42c5-b08d-fb573e39f3a1/resourceGroups/rg-projetoDS/providers/Microsoft.MachineLearningServices/workspaces/workspace-dioDS/codes/20be19b7-486d-4623-b9e0-9314868b6366/versions/1
  creation_context:
    created_at: '2026-02-22T22:19:49.016399+00:00'
    created_by: LUCCA QUEIROZ
    created_by_type: User
  is_deterministic: true
compute: azureml:sorvete
properties:
  _azureml.ComputeTargetType: amlctrain
  _azure

In [7]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

modelo = ml_client.models.create_or_update(
    Model(
        name="GelatomagicoPredictor",
        path=f"azureml://jobs/{returned_job.name}/outputs/artifacts/paths/model/",
        type=AssetTypes.MLFLOW_MODEL,
        description="Previsão de vendas de sorvete por temperatura"
    )
)
print(f" Modelo registrado: versão {modelo.version}")

 Modelo registrado: versão 1
